# Prepare Data for Shiny

In this notebook we prepare the data so that it can be better visualized in an R Shiny application.

## Step 1: Read and Preview the Original File

In [1]:
import pandas as pd

df = pd.read_csv('/project_data/data_asset/Historical_Product_Demand.csv')
df = df.dropna()
df.head()

Product_Code Warehouse Product_Category       Date Order_Demand
0  Product_0993    Whse_J     Category_028  2012/7/27         100 
1  Product_0979    Whse_J     Category_028  2012/1/19         500 
2  Product_0979    Whse_J     Category_028   2012/2/3         500 
3  Product_0979    Whse_J     Category_028   2012/2/9         500 
4  Product_0979    Whse_J     Category_028   2012/3/2         500

In [2]:
df['Order_Demand'] = df['Order_Demand'].str.strip(' ')
df['Order_Demand'] = df['Order_Demand'].str.replace('(', '')
df['Order_Demand'] = df['Order_Demand'].str.replace(')', '')

df['Order_Demand'] = df['Order_Demand'].astype(int)
df.groupby('Warehouse')['Order_Demand'].sum()

Warehouse
Whse_A     147877431
Whse_C     585071404
Whse_J    3363200396
Whse_S    1038024700
Name: Order_Demand, dtype: int64

## Step 2: Prepare Data

### Add Location Information to the Warehouses

In [3]:
def set_wh(x):
    if x == 'Whse_J':
        return 'Los Angeles'
    elif x == 'Whse_A':
        return 'Eugene'
    elif x == 'Whse_S':
        return 'Seattle'
    else:
        return 'Sacramento'

df['Warehouse'] = df['Warehouse'].map(lambda x: set_wh(x))
df['Warehouse'].value_counts()

Los Angeles    764447
Eugene         142335
Seattle         88200
Sacramento      42354
Name: Warehouse, dtype: int64

### Generate Inventory Data

In [4]:
import random
df['Inventory'] = df['Order_Demand'].map(lambda x: random.randint(int(0.8*x), int(1.2*x)))

In [5]:
df['Date'] = pd.to_datetime(df['Date'])
df['Date_Month'] = pd.to_datetime(df['Date'].map(lambda x: str(int(x.year)) + '-' + str(int(x.month)) + '-' + '01'))

### Add Details on State and Region

In [6]:
df_shiny = df.groupby(['Warehouse', 'Product_Code', 'Product_Category', 'Date_Month'])[['Order_Demand', 'Inventory']].sum().reset_index()
def set_state(x):
    if x == 'Los Angeles' or x == 'Sacramento':
        return 'CA'
    elif x == 'Seattle':
        return 'WA'
    else:
        return 'OR'
df_shiny['Warehouse_State'] = df_shiny['Warehouse'].map(lambda x: set_state(x))
df_shiny['Warehouse_Region'] = 'West Coast Region'
df_shiny.head()

Warehouse  Product_Code Product_Category Date_Month  Order_Demand  \
0    Eugene  Product_0001     Category_005 2012-01-01          6400   
1    Eugene  Product_0001     Category_005 2012-02-01         10400   
2    Eugene  Product_0001     Category_005 2012-03-01         11000   
3    Eugene  Product_0001     Category_005 2012-04-01          1500   
4    Eugene  Product_0001     Category_005 2012-05-01          8000   

   Inventory Warehouse_State   Warehouse_Region  
0       6757              OR  West Coast Region  
1      10949              OR  West Coast Region  
2      11382              OR  West Coast Region  
3       1660              OR  West Coast Region  
4       8090              OR  West Coast Region

### Calculate Percent Demand and Create a Binary Label to Indicate if Current Inventory would be Enough to Meet the Demand or Not

In [7]:
df_shiny['pct_demand'] = df_shiny['Inventory'] / df_shiny['Order_Demand']
df_shiny['if_enough'] = (df_shiny['pct_demand'] >= 1).astype(int)

df_shiny.head()

Warehouse  Product_Code Product_Category Date_Month  Order_Demand  \
0    Eugene  Product_0001     Category_005 2012-01-01          6400   
1    Eugene  Product_0001     Category_005 2012-02-01         10400   
2    Eugene  Product_0001     Category_005 2012-03-01         11000   
3    Eugene  Product_0001     Category_005 2012-04-01          1500   
4    Eugene  Product_0001     Category_005 2012-05-01          8000   

   Inventory Warehouse_State   Warehouse_Region  pct_demand  if_enough  
0       6757              OR  West Coast Region    1.055781          1  
1      10949              OR  West Coast Region    1.052788          1  
2      11382              OR  West Coast Region    1.034727          1  
3       1660              OR  West Coast Region    1.106667          1  
4       8090              OR  West Coast Region    1.011250          1

## Step 3: Save the Final File Back to the Project

In [8]:
df_shiny.to_csv('/project_data/data_asset/Monthly_Demand_Inventory.csv', index = False)

**Sample Materials, provided under license. <br>
Licensed Materials - Property of IBM. <br>
© Copyright IBM Corp. 2020 <br>
All Rights Reserved. <br>
US Government Users Restricted Rights - Use, duplication or disclosure restricted by GSA ADP Schedule Contract with IBM Corp.**